# Convert ncs to json

修改地址：dirname 进行测试

'''
{
"CellNumber":,
"SampleFrequency":,
  "Number_of_valid_samples":, 
   "Time_data" : {[Timestamp, Samples],[Timestamp, Samples],...,[Timestamp, Samples]}
}
'''

In [ ]:
import os
import json
import re
from neo.io import NeuralynxIO

def convert_ncs_to_json(filename, input_dirname, output_dirname):
    full_path = os.path.join(input_dirname, filename)
    reader = NeuralynxIO(filename=full_path)
    blk = reader.read_block()
    seg = blk.segments[0]
    signal = seg.analogsignals[0]

    timestamps = [(float(t), float(signal.magnitude[i])) for i, t in enumerate(signal.times.rescale('s').magnitude)]
    sample_frequency = float(signal.sampling_rate.rescale('Hz').magnitude)
    num_valid_samples = int(signal.shape[0])

    data_dict = {
        "CellNumber": re.search(r'CSC(\d+)_FSI20\.ncs', filename).group(1),  # Assuming 'CSC<Number>_FSI20.ncs' format
#         "CellNumber": re.search(r'\d+', filename),
        "SampleFrequency": sample_frequency,
        "Number_of_valid_samples": num_valid_samples,
        "Time_data": timestamps
    }

    json_data = json.dumps(data_dict, indent=4)
    json_filename = os.path.splitext(filename)[0] + '.json'
    json_file_path = os.path.join(output_dirname, json_filename)

    if not os.path.exists(output_dirname):
        os.makedirs(output_dirname)

    with open(json_file_path, 'w') as json_file:
        json_file.write(json_data)

    print(f"JSON file created: {json_filename}")

# Directory settings
input_dirname = "/Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1"
output_dirname = "/Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/ncs_json"

# List all .ncs files
ncs_file_list = [f for f in os.listdir(input_dirname) if f.endswith('.ncs')]

# Process each .ncs file
for ncs_file in ncs_file_list:
    convert_ncs_to_json(ncs_file, input_dirname, output_dirname)

## Read json file example

In [1]:
import json

# Specify the path to the JSON file you want to read
json_filename = r"/Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/ncs_json/CSC6_FSI20.json"

# Read the JSON file
with open(json_filename, 'r') as file:
    data = json.load(file)

feature_names = list(data.keys())
print("Feature Names:", feature_names)

# Display the base information for each key in the JSON file
for key, value in data.items():
    print(f"{key}:")
    if isinstance(value, list):
        # Print only the first few items if the value is a list
        print(value[:5])  # Modify this number based on how many examples you want to see
    else:
        print(value)

Feature Names: ['CellNumber', 'SampleFrequency', 'Number_of_valid_samples', 'Time_data']
CellNumber:
6
SampleFrequency:
1600.0
Number_of_valid_samples:
5579264
Time_data:
[[0.0, 84.53627014160156], [0.000625, 81.78960418701172], [0.00125, 81.05716705322266], [0.001875, 55.11642837524414], [0.0025, 41.07791519165039]]
